Загрузка файлов для аннотации генома после выполнения кода в первом ноутбуке:

In [ ]:
!wget https://raw.githubusercontent.com/DaryaSushkova/hse22_hw2/main/data/gms2.lst
!wget https://raw.githubusercontent.com/DaryaSushkova/hse22_hw2/main/data/proteins.fasta
!wget https://raw.githubusercontent.com/DaryaSushkova/hse22_hw2/main/data/scaffolds.hits_from_MIL_1.txt
!wget https://raw.githubusercontent.com/DaryaSushkova/hse22_hw2/main/data/scaffolds.hits_from_SwissProt.txt

--2022-10-19 19:30:59--  https://raw.githubusercontent.com/DaryaSushkova/hse22_hw2/main/data/gms2.lst
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 221257 (216K) [text/plain]
Saving to: ‘gms2.lst’

gms2.lst            100%[===================>] 216.07K  --.-KB/s    in 0.03s   

2022-10-19 19:30:59 (8.08 MB/s) - ‘gms2.lst’ saved [221257/221257]

--2022-10-19 19:30:59--  https://raw.githubusercontent.com/DaryaSushkova/hse22_hw2/main/data/proteins.fasta
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1404428 (1.3M) [text/plain]
Sav

Также скачиваем данные по близкородственной бактерии T.oleivorans:

In [ ]:
!sh -c "$(curl -fsSL ftp://ftp.ncbi.nlm.nih.gov/entrez/entrezdirect/install-edirect.sh)"


Entrez Direct has been successfully downloaded and installed.


To activate EDirect for this terminal session, please execute the following:

export PATH=${PATH}:${HOME}/edirect



In [ ]:
!$HOME/edirect/efetch -db nuccore -id HF680312 -format gb  >  T_oleivorans_MIL_1.gbk

Устанавливаем библиотеку Biopython и все необходимые модули:

In [ ]:
!pip install biopython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqUtils import GC
from Bio.Data import CodonTable
from Bio import Entrez
from Bio.SeqFeature import SeqFeature, FeatureLocation
import pandas as pd

Аннотация скаффолдов выглядит следующим образом:

In [ ]:
!wget https://raw.githubusercontent.com/DaryaSushkova/hse22_hw1/main/data/scaffolds.fasta

--2022-10-19 19:31:32--  https://raw.githubusercontent.com/DaryaSushkova/hse22_hw1/main/data/scaffolds.fasta
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3977075 (3.8M) [text/plain]
Saving to: ‘scaffolds.fasta’

scaffolds.fasta     100%[===================>]   3.79M  --.-KB/s    in 0.07s   

2022-10-19 19:31:32 (56.1 MB/s) - ‘scaffolds.fasta’ saved [3977075/3977075]



In [ ]:
scaffolds = dict()

for scaffold in SeqIO.parse("scaffolds.fasta", "fasta"):
  scaffold.annotations['molecule_type'] = 'DNA'
  # Добавление аннотированного скаффолда в общий словарь.
  scaffolds[scaffold.id] = scaffold

###Добавляем координаты генов:

In [ ]:
genes = dict()

for gene in SeqIO.parse("proteins.fasta", "fasta"):
  specs = gene.description.split(' ')  # Список характеристик.

  # Базовые данные последовательности для создания объекта SeqFeature.
  scaffold = specs[1]
  start = int(specs[2])
  end = int(specs[3])
  if specs[4] == '+':
    strand = 1
  else:
    strand = -1
  
  curr_gene = SeqFeature(FeatureLocation(start, end, strand = strand), type="CDS")
  curr_gene.qualifiers['locus_tag'] = [specs[0]]
  # Добавление трансляции - последовательности аминокислот.
  curr_gene.qualifiers['translation'] = [gene.seq]
  # Также добавляем информацию в словарь скаффолдов.
  scaffolds[scaffold].features.append(curr_gene)
  genes[specs[0]] = curr_gene

###Добавляем функции белков (из бактерии MIL-1):

In [ ]:
mil_1_genes = dict()

for gene in SeqIO.read("T_oleivorans_MIL_1.gbk", "genbank").features:
  if 'protein_id' not in gene.qualifiers or 'product' not in gene.qualifiers:
    continue
  mil_1_genes[gene.qualifiers['protein_id'][0]] = gene.qualifiers['product'][0]

print(mil_1_genes)

{'CCU70450.1': 'chromosomal replication initiator protein', 'CCU70451.1': 'DNA polymerase III subunit beta', 'CCU70452.1': 'DNA replication and repair protein RecF', 'CCU70453.1': 'DNA gyrase subunit B', 'CCU70454.1': 'hypothetical protein', 'CCU70455.1': 'hypothetical protein', 'CCU70456.1': 'hypothetical protein', 'CCU70457.1': 'hypothetical protein', 'CCU70458.1': 'hypothetical protein', 'CCU70459.1': 'hypothetical protein', 'CCU70460.1': 'transposase orfA', 'CCU70461.1': 'transposase orfB', 'CCU70462.1': 'hypothetical protein', 'CCU70463.1': 'metal-dependent hydrolase', 'CCU70464.1': 'small-conductance mechanosensitive channel-like protein', 'CCU70465.1': 'hypothetical protein', 'CCU70466.1': 'acyltransferase', 'CCU70467.1': 'histidinol phosphatase', 'CCU70468.1': 'glycyl-tRNA synthetase, beta subunit', 'CCU70469.1': 'glycyl-tRNA synthetase, alpha subunit', 'CCU70470.1': 'spermidine synthase', 'CCU70471.1': 'hypothetical protein', 'CCU70472.1': 'alpha/beta hydrolase fold protein', 

С помощью pandas построим таблицу.   
Наименование столбцов и их заполнение исходя из содержания ранее загруженного файла.

In [ ]:
columns = ['qseqid', 'sseqid', 'pident', 'length', 'misMatch', 'gapOpen', 'qstart', 'qend', 'sstart', 'send', 'e-value', 'bitScore']
mil_1_table = pd.read_csv('scaffolds.hits_from_MIL_1.txt', sep='\t', header=None, names=columns)
mil_1_table

,qseqid,sseqid,pident,length,misMatch,gapOpen,qstart,qend,sstart,send,e-value,bitScore
0,1,lcl|HF680312.1_prot_CCU71653.1_1204,98.990,99,1,0,1,99,1,99,8.080000e-68,196.0
1,1,lcl|HF680312.1_prot_CCU72283.1_1834,97.980,99,2,0,1,99,1,99,2.290000e-67,194.0
2,1,lcl|HF680312.1_prot_CCU71933.1_1484,97.980,99,2,0,1,99,1,99,4.150000e-66,191.0
3,1,lcl|HF680312.1_prot_CCU71592.1_1143,97.980,99,2,0,1,99,1,99,4.150000e-66,191.0
4,1,lcl|HF680312.1_prot_CCU71865.1_1416,96.774,62,2,0,1,62,1,62,2.150000e-42,130.0
...,...,...,...,...,...,...,...,...,...,...,...,...
10550,3619,lcl|HF680312.1_prot_1833,100.000,52,0,0,1,52,95,146,5.410000e-34,109.0
10551,3619,lcl|HF680312.1_prot_CCU71654.1_1205,100.000,52,0,0,1,52,95,146,2.250000e-33,110.0
10552,3619,lcl|HF680312.1_prot_CCU71593.1_1144,100.000,52,0,0,1,52,95,146,2.510000e-33,110.0
10553,3619,lcl|HF680312.1_prot_CCU71934.1_1485,100.000,52,0,0,1,52,95,146,4.490000e-33,110.0


Проведем фильтрацию данных (количество строк должно совпасть с ранее найденным 3336):

In [ ]:
cleaned = mil_1_table[mil_1_table['sseqid'].str.contains("CCU")]
cleaned = cleaned.sort_values('bitScore', ascending=False).drop_duplicates('qseqid')
cleaned

,qseqid,sseqid,pident,length,misMatch,gapOpen,qstart,qend,sstart,send,e-value,bitScore
2831,1023,lcl|HF680312.1_prot_CCU73896.1_3447,98.979,2253,23,0,1,2253,1,2253,0.000000e+00,4296.0
4238,1485,lcl|HF680312.1_prot_CCU70724.1_275,97.870,2113,43,1,7,2119,1,2111,0.000000e+00,4158.0
9155,3220,lcl|HF680312.1_prot_CCU71900.1_1451,99.878,1640,2,0,1,1640,1,1640,0.000000e+00,3406.0
2609,947,lcl|HF680312.1_prot_CCU73861.1_3412,99.214,1654,13,0,1,1654,1,1654,0.000000e+00,3372.0
9962,3498,lcl|HF680312.1_prot_CCU71621.1_1172,99.690,1611,5,0,1,1611,1,1611,0.000000e+00,3338.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9235,3263,lcl|HF680312.1_prot_CCU73367.1_2918,41.096,73,43,0,5,77,8,80,7.960000e-13,54.7
10398,3596,lcl|HF680312.1_prot_CCU72347.1_1898,29.592,98,63,2,16,112,4,96,1.250000e-11,53.5
4806,1680,lcl|HF680312.1_prot_CCU72581.1_2132,29.897,97,62,2,8,103,5,96,1.010000e-11,53.5
8063,2817,lcl|HF680312.1_prot_CCU73193.1_2744,29.897,97,62,2,8,103,5,96,1.010000e-11,53.5


Обновляем данные для ранее добавленных генов:

In [ ]:
for index, hit in cleaned.iterrows():
    gene = genes[str(hit['qseqid'])]
    ccu = hit['sseqid'].split('_')[2]
    gene.qualifiers['note'] = ccu
    gene.qualifiers['product'] = [mil_1_genes[ccu]]

SeqIO.write(scaffolds.values(), "GENOME.gbk", "genbank")

69

###Добавляем функции белков (из БД SwissProt)

Получение функций белков SwissProt из файла:

In [ ]:
!wget https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.dat.gz
!gzip -d uniprot_sprot.dat.gz
!grep '^ID\|^DE   RecName: Full=' uniprot_sprot.dat > SwissProt_names.txt

--2022-10-19 19:31:35--  https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.dat.gz
Resolving ftp.uniprot.org (ftp.uniprot.org)... 128.175.240.195
Connecting to ftp.uniprot.org (ftp.uniprot.org)|128.175.240.195|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 642093634 (612M) [application/x-gzip]
Saving to: ‘uniprot_sprot.dat.gz’

uniprot_sprot.dat.g 100%[===================>] 612.35M  41.0MB/s    in 12s     

2022-10-19 19:31:47 (49.7 MB/s) - ‘uniprot_sprot.dat.gz’ saved [642093634/642093634]



Код, который добавляет функции белков из БД SwissProt в итоговую аннотацию:

In [ ]:
prev = None
swiss_genes = dict()

# Формируем словарь для генов.
for swiss in open('SwissProt_names.txt'):
  if swiss.startswith('ID'):
    prev = swiss.split()[1]
  if swiss.startswith('DE'):
    swiss_genes[prev] = swiss.split('=')[1][:-2]

In [ ]:
# Создаем таблицу и фильтруем её.
swiss_table = pd.read_csv('scaffolds.hits_from_SwissProt.txt', sep='\t', header=None, names=columns)
swiss_cleaned = swiss_table.sort_values('bitScore', ascending=False).drop_duplicates('qseqid')

In [ ]:
# Добавляем функции белков в аннотацию.
for index, swiss in swiss_cleaned.iterrows():
  gene = genes[str(swiss['qseqid'])]
  ccu = swiss['sseqid'].split('|')[-1]
  gene.qualifiers['note'] = ccu
  gene.qualifiers['product'] = [swiss_genes[ccu]]

Добалвяем полученную информацию в аннотированный геном:

In [ ]:
SeqIO.write(scaffolds.values(), "GENOME.gbk", "genbank")

69

### Также здесь посчитаем характеристики для таблички ДЗ:

In [ ]:
# Количество скаффолдов.
# Опционально - можно было просто посмотреть в отчет hw1.
general = list(SeqIO.parse("scaffolds.fasta", "fasta"))
len(general)

69

In [ ]:
# Самый длинный скаффолд.
# Опционально - можно было просто посмотреть в отчет hw1.
lens = [len(seq) for seq in general]
max(lens)

3885251

In [ ]:
# Общее количество генов.
len(list(SeqIO.parse("proteins.fasta", "fasta")))

3621